In [1]:
# Aggregate the results and generate a plot
#
# Authors: Parisa Zahedi, Jonathan de Bruin
#
# Dependencies: sklearn, numpy
# License: BSD-3-Clause

# pylint: disable=C0321

import os
import re
import glob
import warnings
import sklearn

from sklearn import metrics
from scipy import stats
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt


In [2]:
dataset_name ="ptsd"
#dataset_name ="ProtonPumpInhibitors"
#dataset_name = "BetaBlockers"
#dataset_name ="CalciumChannelBlockers"
#dataset_name = "Opiods"
#dataset_name = "ACEInhibitors"
#dataset_name = "AtypicalAntipsychotics"
#dataset_name = "Statins"

In [3]:
def get_threshholds(pred):
        desc = stats.describe(pred)
        (min_prob, max_prob) = desc.minmax
        print('min_prob,max_prob', min_prob, max_prob)
        step = (max_prob - min_prob) / 25
        threshholds = [i for i in np.arange(max_prob, min_prob - step, -step)]
        threshholds += [min_prob]
        return threshholds

def get_scores_threshhold(pred, label, threshhold):
        y_classes = ([0 if x < float(threshhold) else 1 for x in pred])
        (tn, fp, fn, tp) = sklearn.metrics.confusion_matrix(label, y_classes).ravel()
        return (tn, fp, fn, tp)
def get_scores(pred, label, allowed_FN):
        threshholds = get_threshholds(pred)
        best_tn, best_fp, best_fn, best_tp = get_scores_threshhold(
            pred, label, threshholds[0])
        ## calculate scores for each threshhold
        for i, threshhold in enumerate(threshholds):
            tn, fp, fn, tp = get_scores_threshhold(
                pred, label, threshhold)
            if (fn <= allowed_FN) and (best_fn > allowed_FN) and (threshhold >0):
                best_tn = tn
                best_fp = fp
                best_fn = fn
                best_tp = tp
            else:  #when fn meets the requirement, we select a smaller threshhold just when it decreases fp.
                if (fn <= allowed_FN) and (best_fp > fp) and (threshhold > 0):
                    best_tn = tn
                    best_fp = fp
                    best_fn = fn
                    best_tp = tp
        return (float(best_tn), float(best_fp), float(best_fn), float(best_tp))

# list(map (lambda lb,pr: get_scores(pr, lb, 1),df.iloc[:,1],df.iloc[:,2]))

def get_score_iterations(df, sample_no,q):
    col_no = len(df.columns)
    scores = list()
    col_names = df.columns.values.tolist()
    for i in np.arange(2,col_no):
        df_ = df.loc[df[col_names[i]] != -1]
        train_size = len(df.loc[df[col_names[i]] == -1])
        (tn,fp,fn,tp)=get_scores(df_.iloc[:,i], df_.iloc[:,1], 1)  
        scores.append([sample_no, i-2, tn,fp,fn,tp,q,train_size])
    return scores

In [ ]:
##Run once
# # store the result
# column_names = ['sample','iteration','tn','fp','fn','tp','q','train_size']

# sample_no =0
# scores= list()

# # save the result to a file
# for q in ('lc','random'):
#     for fp in glob.glob(os.path.join('..','output','active_learning',dataset_name, 'dataset_{}_sr_lstm_active*_q_{}.csv'.format(dataset_name,q))):
#         sample_no +=1 
#         with open(fp) as csv_file:
#             # load the result into memory 
#             df = pd.read_csv(fp)
#         score_sample = get_score_iterations(df, sample_no,q) 
#         scores +=score_sample
        
# scores_df = pd.DataFrame.from_records(scores, columns=column_names)
# scores_df.to_csv(os.path.join('..', 'output','active_learning',dataset_name, 'sr_lstm_results.csv'))

In [4]:
column_names = ['sample','iteration','tn','fp','fn','tp','q','train_size']

for fp in glob.glob(os.path.join('..','output','active_learning',dataset_name, 'dataset_{}_sr_lstm_active1_q_lc.csv'.format(dataset_name))):
        
        with open(fp) as csv_file:
            # load the result into memory 
            df = pd.read_csv(fp)
            print(len(df.loc[df['0'] ==-1]))
            print(len(df.loc[df['1'] ==-1]))
            print(len(df.loc[df['2'] ==-1]))
            print(len(df.loc[df['3'] ==-1]))
            print(len(df.loc[df['4'] ==-1]))
            print(len(df.loc[df['5'] ==-1]))
        score_sample = get_score_iterations(df, 1,'lc') 
scores_test = pd.DataFrame.from_records(score_sample, columns=column_names)
scores_test.to_csv(os.path.join('..', 'output','active_learning',dataset_name, 'sr_lstm_results_test.csv'))        


df_results = pd.read_csv(os.path.join('..', 'output','active_learning',dataset_name, 'sr_lstm_results_test.csv')) #, index_col='sample'

df_results['to_read_plus_train_number'] = (df_results['fp']+df_results['tp']+ df_results['train_size'])
df_results['to_read_plus_train_rate'] = df_results['to_read_plus_train_number']/(df_results['fp']+
                                                                                 df_results['fn']+
                                                                                 df_results['tp']+
                                                                                 df_results['tn']+ 
                                                                                 df_results['train_size'])
df_results

20
30
40
50
60
70
min_prob,max_prob 0.496583133936 0.505677461624
min_prob,max_prob 0.495219141245 0.511256039143
min_prob,max_prob 0.496985942125 0.508712172508
min_prob,max_prob 0.492624551058 0.503949463367
min_prob,max_prob 0.495144784451 0.503873586655
min_prob,max_prob 0.496276408434 0.509455680847
min_prob,max_prob 0.492551058531 0.505178332329
min_prob,max_prob 0.494379937649 0.506409049034
min_prob,max_prob 0.492883086205 0.504222035408
min_prob,max_prob 0.493424147367 0.50517553091
min_prob,max_prob 0.493838965893 0.502781033516
min_prob,max_prob 0.497890144587 0.508709490299
min_prob,max_prob 0.492433428764 0.503160953522
min_prob,max_prob 0.495250791311 0.506226420403
min_prob,max_prob 0.494510680437 0.506276249886
min_prob,max_prob 0.493537306786 0.506632387638
min_prob,max_prob 0.494508773088 0.506257116795
min_prob,max_prob 0.490795999765 0.507692575455
min_prob,max_prob 0.495703846216 0.5064625144
min_prob,max_prob 0.497180849314 0.506054103374
min_prob,max_prob 0.49615

,Unnamed: 0,sample,iteration,tn,fp,fn,tp,q,train_size,to_read_plus_train_number,to_read_plus_train_rate
0,0,1,0,78.0,4949.0,1.0,29.0,lc,20,4998.0,0.984440
1,1,1,1,407.0,4610.0,1.0,29.0,lc,30,4669.0,0.919638
2,2,1,2,66.0,4941.0,1.0,29.0,lc,40,5010.0,0.986803
3,3,1,3,114.0,4883.0,0.0,30.0,lc,50,4963.0,0.977546
4,4,1,4,115.0,4872.0,1.0,29.0,lc,60,4961.0,0.977152
5,5,1,5,280.0,4697.0,0.0,30.0,lc,70,4797.0,0.944849
6,6,1,6,452.0,4515.0,1.0,29.0,lc,80,4624.0,0.910774
7,7,1,7,306.0,4651.0,1.0,29.0,lc,90,4770.0,0.939531
8,8,1,8,320.0,4627.0,1.0,29.0,lc,100,4756.0,0.936774
9,9,1,9,310.0,4627.0,1.0,29.0,lc,110,4766.0,0.938743


In [ ]:
# dataset_name ="BetaBlockers"
init_paper =20
batch_size = 10
df_results = pd.read_csv(os.path.join('..', 'output','active_learning',dataset_name, 'sr_lstm_results.csv')) #, index_col='sample'

df_results['iteration']= df_results['iteration'] -1
df_results['recall'] = df_results['tp']/(df_results['tp']+df_results['fn'])
df_results['fp_rate'] = df_results['fp']/(df_results['fp']+df_results['fn'])

df_results['to_read_plus_train_number'] = (df_results['fp']+df_results['tp']+(df_results['iteration']* batch_size)+init_paper)
df_results['to_read_plus_train_rate'] = df_results['to_read_plus_train_number']/(df_results['fp']+df_results['fn']+df_results['tp']+df_results['tn']+(df_results['iteration']* batch_size)+init_paper)

df_results['to_read_number'] = (df_results['fp']+df_results['tp'])
df_results['to_read_rate'] = df_results['to_read_number']/(df_results['fp']+df_results['fn']+df_results['tp']+df_results['tn'])


In [ ]:
df= df_results.loc[df_results['q']=='random',['sample','iteration','to_read_plus_train_number']]
df_random = df.pivot(index='iteration',columns='sample',values='to_read_plus_train_number')

df_random['mean']=df_random.mean(axis=1)


df= df_results.loc[df_results['q']=='lc',['sample','iteration','to_read_plus_train_number']]
df_lc = df.pivot(index='iteration',columns='sample',values='to_read_plus_train_number')
df_lc['mean']=df_lc.mean(axis=1)

df_lc.loc[[0,48],:]

## Visualize training size and initial included papers

In [ ]:
%matplotlib inline

df_lc.max().sort_values()
df_lc

In [ ]:
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-darkgrid')
my_dpi=96
plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
# multiple line plot
for column in df_random:
        plt.plot(df_random.index, df_random[column], marker='', color='gray', linewidth=1, alpha=0.4)

plt.plot(df_random.index, df_random['mean'], marker='', color= 'orange', linewidth=4, alpha=0.7)
plt.xlim(0,50)
plt.xlabel('iteration')
plt.ylabel('Number of papers to read')
plt.title('Number of papers to read -random')



plt.show()

###################
plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)

for column in df_lc:
        plt.plot(df_lc.index, df_lc[column], marker='', color='gray',  linewidth=1, alpha=0.4)

plt.plot(df_lc.index, df_lc['mean'], marker='', color= 'orange', linewidth=4, alpha=0.7)

plt.xlim(0,50)
plt.xlabel('iteration')
plt.ylabel('Number of papers to read')
plt.title('Number of papers to read -lc')

plt.show()


In [ ]:
# selected_col_names = ['training_size','allowed_FN','init_included_papers','dataset','fp_rate','recall','to_read_number','to_read_rate','tn','fp','fn','tp']
df_results = pd.read_csv(os.path.join('..', 'output','passive',dataset_name, 'ptsd_results.csv'))
    
df_results['to_read_plus_train_number'] = (df_results['fp']+df_results['tp']+df_results['training_size'])
df_results['to_read_plus_train_rate'] = df_results['to_read_plus_train_number']/(df_results['fp']+df_results['fn']+df_results['tp']+df_results['tn']+df_results['training_size'])

cond = (df_results['allowed_FN'] == 1) & (df_results['dropout'] == 0.4) & (df_results['init_included_papers'] == 10)
df_pas = df_results.loc[cond,]
df_pas

fig, ax = plt.subplots(figsize=(12, 7))
m = df_pas.groupby(['training_size','init_included_papers']).mean()['to_read_plus_train_number'].unstack()
st = df_pas.groupby(['training_size','init_included_papers']).std()['to_read_plus_train_number'].unstack()
min_ = df_pas.groupby(['training_size','init_included_papers']).min()['to_read_plus_train_number'].unstack()


s=10
ax.plot(m.index, m[s], 'r', label=s)
ax.fill_between(m.index, m[s] - st[s], m[s] + st[s], color='r', alpha=0.15)
ax.plot(m.index, min_[s], "{}--".format('r', alpha=0.5, label="min of {}".format(s)))
plt.show()






#     if not os.path.isdir(os.path.join('..', 'output', 'figures', dataset_name)):
#         os.mkdir(os.path.join('..', 'output', 'figures', dataset_name))
#     fig.savefig(os.path.join('..','output', "figures", dataset_name, "perf_plot-{}_{}.png".format(r,dataset_name)))

In [ ]:
dp_path= os.path.join("..","data","datasets","depression_review","adults_depression.csv")
dp_df = pd.read_csv(dp_path, encoding='latin-1')

In [ ]:
dval = [[0.1,0.5],[0.3,0.7],[0.6,0.5]]